In [121]:
import pandas as pd
import requests
import os
from datetime import datetime, date, timedelta
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np

# Fred API Calls

In [122]:
#API key for FRED
API_KEY = '4f1ae905f4e737c1025e457379bd360a'

#Base URL for FRED API
BASE_URL = 'https://api.stlouisfed.org/fred/'


In [123]:
def fetch_fred_data(series_id, start_date, end_date, frequency, aggregation_method):
    # FRED API URL
    url = f'{BASE_URL}series/observations'

    # Parameters for the API request
    params = {
        'series_id': series_id,
        'api_key': API_KEY,
        'file_type': 'json',
        'frequency': frequency,
        'aggregation_method': aggregation_method,
        'observation_start': start_date,
        'observation_end': end_date,
    }

    # Sending the request to the FRED API
    response = requests.get(url, params=params)
    data = response.json()

    # Check for errors
    if 'observations' not in data:
        print(f"Error fetching data for {series_id}: {data.get('error_message', 'Unknown error')}")
        return pd.DataFrame()

    # Extract observations
    observations = data['observations']
    
    # Convert to DataFrame
    df = pd.DataFrame(observations)
    df['date'] = pd.to_datetime(df['date'])
    df['value'] = pd.to_numeric(df['value'], errors='coerce')  # Convert values to numeric

    # Select relevant columns
    df = df[['date', 'value']]
    
    return df


In [124]:
#Define Date Range
end_date = date.today().replace(day=1) - timedelta(days=1)
start_date = (end_date - timedelta(days=20*365.24)).replace(day=1)
print(start_date, end_date)

2004-12-01 2024-11-30


In [125]:
#Fetch Unemployment Rate (UNRATE) data
unemployment_series_id = 'UNRATE' 

# Fetch data
fetched_unemployment_data = fetch_fred_data(unemployment_series_id, start_date, end_date,"m",'avg')


In [126]:
#Fetch Fed Rate (FEDFUNDS) data
fed_series_id = 'FEDFUNDS' 

# Fetch data
fetched_fed_data = fetch_fred_data(fed_series_id, start_date, end_date,"m",'avg')


In [127]:
#Fetch Consumer Price Index for All Urban Consumers: All Items in U.S. City Average (CPIAUSCL) data from 2020-01-01 to 2024-01-01
CPI_series_id = 'CPIAUCSL'

# Fetch data
fetched_cpi_data = fetch_fred_data(CPI_series_id, start_date, end_date,"m",'avg')

In [128]:
#Fetch BBB ICE BofA Index Effective Yield
BBB_series_id = 'BAMLC0A4CBBBEY'

# Fetch data
fetched_BBB_data = fetch_fred_data(BBB_series_id, start_date, end_date,"m",'avg')

In [129]:
#Fetch SP500 Index
# sp500_series_id = 'SP500'

# # Fetch data
# fetched_sp500_data = fetch_fred_data(sp500_series_id, start_date, end_date,"d",'eop')

In [130]:
start_date

datetime.date(2004, 12, 1)

In [131]:
# fetched_sp500_data

In [132]:
#Fetch VIX Data
vix_series_id = 'VIXCLS'

#Fetch historical market data
fetched_vix_data = fetch_fred_data(vix_series_id, start_date, end_date,"d",'eop')


In [133]:
fetched_vix_data

,date,value
0,2004-12-01,12.97
1,2004-12-02,12.98
2,2004-12-03,12.96
3,2004-12-06,13.19
4,2004-12-07,13.67
...,...,...
5213,2024-11-25,14.60
5214,2024-11-26,14.10
5215,2024-11-27,14.10
5216,2024-11-28,13.90


In [134]:
#Fetch GDP Data
gdp_series_id = 'GDP'

#Fetch historical GDP data
fetched_gdp_data = fetch_fred_data(gdp_series_id, start_date, end_date,"q","avg")


In [135]:
#Fetch JPY/USD Exchange Rate
jpy_usd_exchange_rate_series_id = 'DEXJPUS'

#Fetch JPY/USD Historical Data
fetched_jpy_usd_exchange_rate_data = fetch_fred_data(jpy_usd_exchange_rate_series_id,start_date,end_date,"m","avg")

In [136]:
#Fetch Oil Prices
oilprices_series_id = 'WTISPLC'

#Fetch Oil Prices Historical Data
fetched_oilprices_data = fetch_fred_data(oilprices_series_id,start_date,end_date,"m","avg")

In [137]:
#Data Cleaning
unemployment_data = fetched_unemployment_data.set_index('date')
unemployment_data.index = pd.to_datetime(unemployment_data.index)
unemployment_data = unemployment_data.rename(columns={'value': 'Unemployment Rate'})

fed_data = fetched_fed_data.set_index('date')
fed_data.index = pd.to_datetime(fed_data.index)
fed_data = fed_data.rename(columns={'value': 'Fed Rate'})

cpi_data = fetched_cpi_data.set_index('date')
cpi_data.index = pd.to_datetime(cpi_data.index)
cpi_data = cpi_data.rename(columns={'value': 'CPI'})
last_cpi_value = cpi_data['CPI'].iloc[-1]
last_cpi_date = cpi_data.index[-1]
new_date_range = pd.date_range(start=last_cpi_date + pd.DateOffset(months = 1), end=end_date + pd.DateOffset(months = 1), freq='ME')
extended_df = pd.DataFrame({'CPI': [last_cpi_value] * len(new_date_range)}, index=new_date_range)
extended_df.index.name = 'date'
cpi_data = cpi_data.resample('ME').ffill()
cpi_data = pd.concat([cpi_data, extended_df])
cpi_data.index = cpi_data.index.to_period('M').to_timestamp()

# threemonth_tbill_data = fetched_threemonth_tbill_data.set_index('date')
# threemonth_tbill_data.index = pd.to_datetime(threemonth_tbill_data.index)
# threemonth_tbill_data = threemonth_tbill_data.rename(columns={'value': 'Three Month T Bill Rate'})

# sixmonth_tbill_data = fetched_sixmonth_tbill_data.set_index('date')
# sixmonth_tbill_data.index = pd.to_datetime(sixmonth_tbill_data.index)
# sixmonth_tbill_data = sixmonth_tbill_data.rename(columns={'value': 'Six Month T Bill Rate'})

# tenyear_treasury_data = fetched_tenyear_treasury_data.set_index('date')
# tenyear_treasury_data.index = pd.to_datetime(tenyear_treasury_data.index)
# tenyear_treasury_data = tenyear_treasury_data.rename(columns={'value': '10-Year Treasury Constant Maturity Rate'})

BBB_data = fetched_BBB_data.set_index('date')
BBB_data.index = pd.to_datetime(BBB_data.index)
BBB_data = BBB_data.rename(columns={'value': 'BBB Rate'})

# AAA_data = fetched_AAA_data.set_index('date')
# AAA_data.index = pd.to_datetime(AAA_data.index)
# AAA_data = AAA_data.rename(columns={'value': 'AAA Rate'})

# BAA_data = fetched_BAA_data.set_index('date')
# BAA_data.index = pd.to_datetime(BAA_data.index)
# BAA_data = BAA_data.rename(columns={'value': 'BAA Rate'})

# sp500_data = fetched_sp500_data.set_index('date')
# sp500_data.index = pd.to_datetime(sp500_data.index)
# sp500_data = sp500_data.rename(columns={'value': 'SP500 Index'})
# sp500_data = sp500_data.resample('ME').median()
# ###sp500_data = sp500_data.diff()
# sp500_data.index = sp500_data.index.to_period('M').to_timestamp()

VIX_data = fetched_vix_data.set_index('date')
VIX_data.index = pd.to_datetime(VIX_data.index)
VIX_data = VIX_data.rename(columns={'value': 'VIX'})
VIX_data = VIX_data.resample('ME').median()
VIX_data.index = VIX_data.index.to_period('M').to_timestamp()

gdp_data = fetched_gdp_data.set_index('date')
gdp_data.index = pd.to_datetime(gdp_data.index)
gdp_data = gdp_data.rename(columns={'value': 'GDP'})
last_gdp_value = gdp_data['GDP'].iloc[-1]
last_gdp_date = gdp_data.index[-1]
new_date_range = pd.date_range(start=last_gdp_date + pd.DateOffset(months = 1), end=end_date + pd.DateOffset(months = 1), freq='ME')
extended_df = pd.DataFrame({'GDP': [last_gdp_value] * len(new_date_range)}, index=new_date_range)
extended_df.index.name = 'date'
gdp_data = gdp_data.resample('ME').ffill()
gdp_data = pd.concat([gdp_data, extended_df])
gdp_data.index = gdp_data.index.to_period('M').to_timestamp()

# initial_claims_data = fetched_initial_claims_data.set_index('date')
# initial_claims_data.index = pd.to_datetime(initial_claims_data.index)
# initial_claims_data = initial_claims_data.rename(columns={'value': 'Initial Claims'})

jpy_usd_exchange_rate_data = fetched_jpy_usd_exchange_rate_data.set_index('date')
jpy_usd_exchange_rate_data.index = pd.to_datetime(jpy_usd_exchange_rate_data.index)
jpy_usd_exchange_rate_data = jpy_usd_exchange_rate_data.rename(columns={'value': 'JPY/USD Spot Rate'})

# nonfarm_payroll_data = fetched_nonfarm_payroll_data.set_index('date')
# nonfarm_payroll_data.index = pd.to_datetime(nonfarm_payroll_data.index)
# nonfarm_payroll_data = nonfarm_payroll_data.rename(columns={'value': 'Total Nonfarm Employees'})

oilprices_data = fetched_oilprices_data.set_index('date')
oilprices_data.index = pd.to_datetime(oilprices_data.index)
oilprices_data = oilprices_data.rename(columns={'value': 'Spot Crude Oil Price'})

In [138]:
cpi_data

,CPI
date,
2004-12-01,191.700
2005-01-01,191.600
2005-02-01,192.400
2005-03-01,193.100
2005-04-01,193.700
...,...
2024-07-01,313.534
2024-08-01,314.121
2024-09-01,314.686


In [139]:
oilprices_data

,Spot Crude Oil Price
date,
2004-12-01,43.33
2005-01-01,46.84
2005-02-01,47.97
2005-03-01,54.31
2005-04-01,53.04
...,...
2024-07-01,81.80
2024-08-01,76.68
2024-09-01,70.24


In [140]:
# Merge dataframes together one by one
merged_data = pd.merge(unemployment_data, cpi_data, on='date')
# merged_data = pd.merge(merged_data, threemonth_tbill_data, on='date')
# merged_data = pd.merge(merged_data, sixmonth_tbill_data, on='date')
# merged_data = pd.merge(merged_data, tenyear_treasury_data, on='date')
merged_data = pd.merge(merged_data, fed_data, on='date')
merged_data = pd.merge(merged_data, BBB_data, on='date')
# merged_data = pd.merge(merged_data, sp500_data, on='date')
merged_data = pd.merge(merged_data, VIX_data, on='date')
merged_data = pd.merge(merged_data, gdp_data, on='date')
# merged_data = pd.merge(merged_data, initial_claims_data, on='date')
merged_data = pd.merge(merged_data, jpy_usd_exchange_rate_data, on='date')
# merged_data = pd.merge(merged_data, nonfarm_payroll_data, on='date')
merged_data = pd.merge(merged_data, oilprices_data, on='date')
merged_data.index.name = "Date"


In [141]:
merged_data.to_csv("VIX_FRED_data_new.csv")

In [142]:
merged_data

,Unemployment Rate,CPI,Fed Rate,BBB Rate,VIX,GDP,JPY/USD Spot Rate,Spot Crude Oil Price
Date,,,,,,,,
2004-12-01,5.4,191.700,2.16,5.04,12.550,12527.214,103.81,43.33
2005-01-01,5.3,191.600,2.28,5.13,13.340,12767.286,103.34,46.84
2005-02-01,5.4,192.400,2.50,5.11,11.600,12767.286,104.94,47.97
2005-03-01,5.2,193.100,2.63,5.50,13.145,12767.286,105.25,54.31
2005-04-01,5.2,193.700,2.79,5.65,14.530,12922.656,107.19,53.04
...,...,...,...,...,...,...,...,...
2024-07-01,4.3,313.534,5.33,5.53,13.190,29374.914,157.52,81.80
2024-08-01,4.2,314.121,5.33,5.20,16.690,29374.914,146.26,76.68
2024-09-01,4.1,314.686,5.13,4.96,17.070,29374.914,142.95,70.24


# YFinance API Calls

In [143]:
sp500 = yf.download('^GSPC', start=start_date, end=end_date, interval='1mo').add_prefix('SP500 ')
nasdaq = yf.download('^IXIC', start=start_date, end=end_date, interval='1mo').add_prefix('NASDAQ ')
dji = yf.download('^DJI', start=start_date, end=end_date, interval='1mo').add_prefix('DJI ')
russ2000 = yf.download('^RUT', start=start_date, end=end_date, interval='1mo').add_prefix('RUSSELL2000 ')
ftse = yf.download('^FTSE', start=start_date, end=end_date, interval='1mo').add_prefix('FTSE ')
nikkei = yf.download('^N225', start=start_date, end=end_date, interval='1mo').add_prefix('NIKKEI ')
gold = yf.download('GC=F', start=start_date, end=end_date, interval='1mo').add_prefix('GOLD ')
usd_index = yf.download('DX-Y.NYB', start=start_date, end=end_date, interval='1mo').add_prefix('USD INDEX ')
# vxn = yf.download('^VXN', start=start_date, end=end_date, interval='1mo').add_prefix('VXN ')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [144]:
gold

Price,GOLD Close,GOLD High,GOLD Low,GOLD Open,GOLD Volume
Ticker,GOLD GC=F,GOLD GC=F,GOLD GC=F,GOLD GC=F,GOLD GC=F
Date,,,,,
2004-12-01,437.500000,456.500000,432.600006,451.600006,12728
2005-01-01,421.799988,431.000000,418.700012,431.000000,115342
2005-02-01,436.500000,436.000000,410.100006,422.200012,8468
2005-03-01,428.700012,446.399994,424.700012,435.000000,3171
2005-04-01,435.000000,438.200012,422.700012,428.299988,2579
...,...,...,...,...,...
2024-06-01,2327.699951,2385.100098,2285.399902,2322.600098,7958
2024-07-01,2426.500000,2473.100098,2321.699951,2323.800049,155434


In [145]:
sp500

Price,SP500 Close,SP500 High,SP500 Low,SP500 Open,SP500 Volume
Ticker,SP500 ^GSPC,SP500 ^GSPC,SP500 ^GSPC,SP500 ^GSPC,SP500 ^GSPC
Date,,,,,
2004-12-01,1211.920044,1217.329956,1173.780029,1173.780029,31102500000
2005-01-01,1181.270020,1217.800049,1163.750000,1211.920044,31498800000
2005-02-01,1203.599976,1212.439941,1180.949951,1181.270020,29297410000
2005-03-01,1180.589966,1229.109985,1163.689941,1203.599976,39014150000
2005-04-01,1156.849976,1191.880005,1136.150024,1180.589966,43424270000
...,...,...,...,...,...
2024-07-01,5522.299805,5669.669922,5390.950195,5471.080078,80160390000
2024-08-01,5648.399902,5651.620117,5119.259766,5537.839844,81097300000


In [146]:
sp500 = sp500[['SP500 Close','SP500 Volume']]
nasdaq=nasdaq[['NASDAQ Close', 'NASDAQ Volume']]
dji = dji[['DJI Close','DJI Volume']]
russ2000 = russ2000[['RUSSELL2000 Close', 'RUSSELL2000 Volume']]
ftse = ftse[['FTSE Close', 'FTSE Volume']]
nikkei = nikkei[['NIKKEI Close', 'NIKKEI Volume']]
gold = gold[['GOLD Volume']]
usd_index = usd_index[['USD INDEX Close']]

In [147]:
merged_yf_data = pd.merge(sp500, nasdaq, on='Date')
merged_yf_data = pd.merge(merged_yf_data, dji, on='Date')
merged_yf_data = pd.merge(merged_yf_data, russ2000, on='Date')
merged_yf_data = pd.merge(merged_yf_data, ftse, on='Date')
merged_yf_data = pd.merge(merged_yf_data, nikkei, on='Date')
merged_yf_data = pd.merge(merged_yf_data, gold, on='Date')
merged_yf_data = pd.merge(merged_yf_data, usd_index, on='Date')
# merged_yf_data = pd.merge(merged_yf_data, vxn, on='Date')

In [148]:
merged_yf_data.columns = merged_yf_data.columns.get_level_values(-1)
merged_yf_data = pd.merge(merged_yf_data, merged_data['VIX'], on='Date')

In [149]:
merged_yf_data

,SP500 ^GSPC,SP500 ^GSPC,NASDAQ ^IXIC,NASDAQ ^IXIC,DJI ^DJI,DJI ^DJI,RUSSELL2000 ^RUT,RUSSELL2000 ^RUT,FTSE ^FTSE,FTSE ^FTSE,NIKKEI ^N225,NIKKEI ^N225,GOLD GC=F,USD INDEX DX-Y.NYB,VIX
Date,,,,,,,,,,,,,,,
2004-12-01,1211.920044,31102500000,2175.439941,44379920000,10783.009766,5744730000,651.570007,31102700000,4814.299805,28451048500,11488.759766,1437800000,12728,80.849998,12.550
2005-01-01,1181.270020,31498800000,2062.409912,43135970000,10489.940430,5424930000,624.020020,31498000000,4852.299805,31489218700,11387.589844,1616200000,115342,83.599998,13.340
2005-02-01,1203.599976,29297410000,2051.719971,36633150000,10766.230469,5038600000,634.059998,29297410000,4968.500000,33004713300,11740.599609,1665700000,8468,82.510002,11.600
2005-03-01,1180.589966,39014150000,1999.229980,39780380000,10503.759766,5820160000,615.070007,39014150000,4894.399902,34739240300,11668.950195,2032200000,3171,84.059998,13.145
2005-04-01,1156.849976,43424270000,1921.650024,37517930000,10192.509766,5852240000,579.380005,43424270000,4801.700195,32554037300,11008.900391,1675000000,2579,84.430000,14.530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-01,5460.479980,76025620000,17732.599609,107772060000,39118.859375,7213580000,2047.689941,76025620000,8164.100098,18910547600,39583.078125,2209000000,7958,105.870003,12.645
2024-07-01,5522.299805,80160390000,17599.400391,119005630000,40842.789062,7507140000,2254.479980,80160390000,8368.000000,16067728100,39101.820312,2570500000,155434,104.099998,13.190
2024-08-01,5648.399902,81097300000,17713.619141,121239310000,41563.078125,8368220000,2217.629883,81097300000,8376.599609,18173603500,38647.750000,3034000000,12103,101.699997,16.690


In [150]:
merged_yf_data.to_csv("VIX_YF_data_new.csv")

In [151]:
all_data = pd.merge(merged_data, merged_yf_data, on='Date')

In [152]:
all_data

,Unemployment Rate,CPI,Fed Rate,BBB Rate,VIX_x,GDP,JPY/USD Spot Rate,Spot Crude Oil Price,SP500 ^GSPC,SP500 ^GSPC,...,DJI ^DJI,RUSSELL2000 ^RUT,RUSSELL2000 ^RUT,FTSE ^FTSE,FTSE ^FTSE,NIKKEI ^N225,NIKKEI ^N225,GOLD GC=F,USD INDEX DX-Y.NYB,VIX_y
Date,,,,,,,,,,,,,,,,,,,,,
2004-12-01,5.4,191.700,2.16,5.04,12.550,12527.214,103.81,43.33,1211.920044,31102500000,...,5744730000,651.570007,31102700000,4814.299805,28451048500,11488.759766,1437800000,12728,80.849998,12.550
2005-01-01,5.3,191.600,2.28,5.13,13.340,12767.286,103.34,46.84,1181.270020,31498800000,...,5424930000,624.020020,31498000000,4852.299805,31489218700,11387.589844,1616200000,115342,83.599998,13.340
2005-02-01,5.4,192.400,2.50,5.11,11.600,12767.286,104.94,47.97,1203.599976,29297410000,...,5038600000,634.059998,29297410000,4968.500000,33004713300,11740.599609,1665700000,8468,82.510002,11.600
2005-03-01,5.2,193.100,2.63,5.50,13.145,12767.286,105.25,54.31,1180.589966,39014150000,...,5820160000,615.070007,39014150000,4894.399902,34739240300,11668.950195,2032200000,3171,84.059998,13.145
2005-04-01,5.2,193.700,2.79,5.65,14.530,12922.656,107.19,53.04,1156.849976,43424270000,...,5852240000,579.380005,43424270000,4801.700195,32554037300,11008.900391,1675000000,2579,84.430000,14.530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-01,4.1,313.049,5.33,5.63,12.645,29016.714,157.86,79.77,5460.479980,76025620000,...,7213580000,2047.689941,76025620000,8164.100098,18910547600,39583.078125,2209000000,7958,105.870003,12.645
2024-07-01,4.3,313.534,5.33,5.53,13.190,29374.914,157.52,81.80,5522.299805,80160390000,...,7507140000,2254.479980,80160390000,8368.000000,16067728100,39101.820312,2570500000,155434,104.099998,13.190
2024-08-01,4.2,314.121,5.33,5.20,16.690,29374.914,146.26,76.68,5648.399902,81097300000,...,8368220000,2217.629883,81097300000,8376.599609,18173603500,38647.750000,3034000000,12103,101.699997,16.690


In [153]:
all_data = all_data.drop(columns=['VIX_y'])
all_data = all_data.rename(columns={'VIX_x':'VIX'})

In [154]:
all_data.to_csv("VIX_data_new.csv")